## map average precision @12

### 1. load package and data

In [1]:
import numpy as np 
import pandas as pd 

In [5]:
from IPython.display import display
from io import BytesIO

In [6]:
# clicks_train 불러오기
%gcs read --object gs://capstone-01/data/clicks_train.csv --variable test_file
df_train = pd.read_csv(BytesIO(test_file))

### 2. create ad_likelihood
여기에서는 클릭할 확률의 추정값으로 ad_likelihood를 구했다 이 점을 참고하자

In [7]:
ad_likelihood = df_train.groupby('ad_id').clicked.agg(['count','sum']).reset_index()
M             = df_train.clicked.mean() # 평균 클릭률 : 0.19364

In [10]:
ad_likelihood.head(10)

,ad_id,count,sum
0,1,2,0
1,2,22,1
2,3,161,11
3,4,32,4
4,5,1,0
5,6,1,0
6,7,3406,985
7,8,28,4
8,9,3,0
9,10,81,9


In [11]:
ad_likelihood['likelihood'] = (ad_likelihood['sum'] + 12*M) / (12 + ad_likelihood['count']) # ad_likelihood의 정의

In [12]:
ad_likelihood.head(10)

,ad_id,count,sum,likelihood
0,1,2,0,0.165982
1,2,22,1,0.097757
2,3,161,11,0.077016
3,4,32,4,0.143721
4,5,1,0,0.178750
5,6,1,0,0.178750
6,7,3406,985,0.288860
7,8,28,4,0.158094
8,9,3,0,0.154916
9,10,81,9,0.121761


In [19]:
df_train = df_train.merge(ad_likelihood, how='left')
df_train.likelihood.fillna(M, inplace=True)

In [20]:
# sort_values가 핵심! likelihood 값을 기준으로 내림차순 정렬했기 때문에, 결국 display_id마다 클릭할 확률이 높은 순서대로 정렬된다.
df_train.sort_values(['display_id', 'likelihood'], inplace=True, ascending=[True, False] ) 

In [21]:
from ml_metrics import mapk

Y_ads = df_train[ df_train.clicked == 1 ].ad_id.values.reshape(-1,1) 
P_ads = df_train.groupby(by='display_id', sort=False).ad_id.apply( lambda x: x.values ).values

# mapk 함수
score = mapk( Y_ads, P_ads, 12 )
print("MAP: %.12f" % score)

MAP: 0.648882425045


In [22]:
Y_ads # 클릭된 ad_id의 집합

array([[144739],
       [308455],
       [228657],
       ...,
       [127047],
       [153244],
       [151498]])

In [23]:
P_ads # display_id마다 클릭할 확률이 높은 순서대로 나열됨

array([array([144739, 279295,  42337, 139684, 296965, 156824]),
       array([308455, 156535, 125211, 169564]),
       array([ 95814, 183846, 152141, 228657,  71547, 250082]), ...,
       array([127047, 397506, 285839, 483448,  36108, 145601]),
       array([153244, 167294, 186600,  82851]),
       array([282350, 521828, 151498, 522693])], dtype=object)